### Background

Population prevalence estimation assumes that the null hypothesis, $H_0$, is true in some of the population and alternative hypothesis, $H_1$, is true in others. In other words, we are modeling our observations as coming from a _mixture_ of two distributions: an $H_0$ distribution (ie. a uniform distribution of $p$-values) and an $H_1$ distribution (a left-skewed distribution).

An advantage of taking a Bayesian approach to this problem is that each of these distributions ($H_0$, $H_1$, and mixture) is a _model_, and a host of Bayesian model comparison tools can be applied to ask which of these three models explains our observed data the best. That means, unlike with frequentist prevalence estimation methods, we get to check our assumptions about whether the prevalence model is valid. 

The `p2prev.PCurveMixture` class has a built-in `compare` method to compare the $p$-Curve Mixture model to $H_0$. `p2prev.PCurveMixture.compare` uses [Pareto smoothed importance sampling leave-one-out cross-validation](dx.doi.org/10.1111/1467-9868.00353) to compare models. (See the `arviz` [documentation](https://python.arviz.org/en/stable/api/generated/arviz.compare.html) for exact methods used, and `prevalence-estimation.ipynb` for a usage example.) The end result is a "weight" assigned to each model, with all the model weights adding up to 1. These weights are like a posterior belief that the model will best describe new data (among the models condidered) and are often interpreted as a probability of a model given the observed data, but this interpretation is quite loose. The model weights technically have more to do with how well the model is likely to predict new data, rather than a posterior belief that the model is _true_ (or, in any event, resembles the actual data-generating process more than the other models considered). 

While the leave-one-out approach is often favored by "full" Bayesians, many practioners prefer __Bayes Factors__, which reflect the _marginal likelihood_ of the observed data given each model (a.k.a. $\mathbb{P}(\text{data}|\text{model})$). The likelihood ratio between models ($\mathbb{P}(\text{data}|\text{model}_i) / \mathbb{P}(\text{data}|\text{model}_j) $) is the factor by which prior beliefs about the models are updated given new data according to Bayes' Rule. Bayes factors are often [misused](http://daniellakens.blogspot.com/2023/04/preventing-common-misconceptions-about.html) but are nonetheless useful for model comparison and are likely to be more familiar (and interpretable) to the average scientist than other methods of Bayesian model comparison.

Unfortunately, computing Bayes Factors is actually quite hard, as the $\mathbb{P}(\text{data}|\text{model})$ is usually intractable to calculate exactly and quite difficult to approximate. 

> Note: More precisely, it's hard to estimate Bayes Factors _well_. Some methods for estimating Bayes Factors [just don't work that well in practice](https://radfordneal.wordpress.com/2008/08/17/the-harmonic-mean-of-the-likelihood-worst-monte-carlo-method-ever/) while many implementations (including common R packages) require strong assumptions about the shape of the posterior or force you to specify priors on $\mathbb{P(\text{data}|\text{model})}$ in a certain parametric form (and Bayes factors are quite sensitive to such priors). The method we use in this notebook, in contrast, is fully non-parametric. 

It's possible to estimate a Bayes factor for the $p$-curve mixture model, which is what we'll do in this notebook. The method is sound but requires care -- Monte Carlo sampling from the posterior is slow and sometimes fails -- and it actually requires us to modify our model implementation (the code, not the math). So Bayes Factors are unlikely to be incorporated into the `p2prev.PCurveMixture` interface anytime soon. 

### How to do it

We'll take the following approach:

> "Computation of Bayes factors can be framed as a hierarchical model, where the high-level parameter is an index assigned to each model and sampled from a categorical distribution. In other words, we perform inference for two (or more) competing models at the same time and we use a discrete dummy variable [$k$] that jumps between models. How much time we spend sampling each model is proportional to [$\mathbb{P}(\text{model}_k|\text{data})$]." - [PyMC documentation](https://www.pymc.io/projects/examples/en/latest/diagnostics_and_criticism/Bayes_factor.html#using-a-hierarchical-model)

This is why we have to re-code our prevalence model to estimate Bayes Factors -- we have to incorporate it into an even larger model, which contains $H_0$, $H_1$, and our $H_0$/$H_1$ mixture as possible (but mutually exclusive) explanations for the data.

Actually implementing this approach in `PyMC` ends up being kind of hacky, but it's possible. Here's my implementation below, but I'd love to hear if anyone has a clever idea for marginalizing out the discrete variable (which would improve sampling immensely):

In [1]:
# if running from 'examples' directory of p2prev repo, 
# we need to add parent directory to module search path
# so that p2prev module can be imported
# (not necessary if you've pip installed p2prev module)
import sys
import os
cwd = os.getcwd()
pardir = os.path.abspath(os.path.join(cwd, os.pardir))
sys.path.append(pardir)

In [2]:
import pymc as pm
import arviz as az
import numpy as np
from p2prev.model import p_curve_loglik

# our observed p-values from 4, n=1 experiments
pvals = [0.00060, 0.02999, 0.04939, 0.94601]
pvals = np.array(pvals) 

with pm.Model() as combined_model:

    # prior for effect size scale under H1
    prior_mean = 1.5 
    # prior on prevalence of H1 under mixture model
    prev = pm.Uniform('prevalence_mix', 0, 1)

    ## all H0 model:
    #------------------------
    pcurve_H0 = pm.Uniform.dist(0, 1)
    H0_logp = pm.logp(pcurve_H0, pvals) # unsampled log-likelihood

    ## all H1 model:
    #------------------------
    delta_H1 = pm.Exponential('effect_size_H1', 1/prior_mean)
    pcurve_H1 = pm.CustomDist.dist(delta_H1, logp = p_curve_loglik)
    H1_logp = pm.logp(pcurve_H1, pvals) # unsampled log-likelihood

    ## H0/H1 mixture model:
    #------------------------
    # need to re-specify the effect size prior (even though
    # prior is same as H1 model) since parameters must
    # not be shared across models during sampling
    delta_mix = pm.Exponential('effect_size_mix', 1/prior_mean)
    pcurve_H1 = pm.CustomDist.dist(delta_mix, logp = p_curve_loglik)
    # but H0 model has no parameters so we can reuse pcurve_H0
    pcurve_mix = pm.Mixture.dist(
        w = [1 - prev, prev], 
        comp_dists = [pcurve_H0, pcurve_H1], 
    )
    mix_logp = pm.logp(pcurve_mix, pvals) # unsampled log-likelihood

    ## Combine into one, hierarchical model:
    #------------------------------------------------
    # we'll put a flat prior over models, so posterior
    # probability ratio will equal the Bayes Factor 
    prior_over_models = np.ones(3)/3 # i.e. P(model_k) = 1/3 
    m = pm.Multinomial('H_true', n = 1, p = prior_over_models)
    # the following isn't a proper pymc likelihood 
    is_H0, is_H1, is_mix = m[0], m[1], m[2]
    logp = (is_H0 * H0_logp).sum() + \
        (is_H1 * H1_logp).sum() + \
        (is_mix * mix_logp).sum()
    # but it will behave like one if we evaluate it in a Potential
    potential = pm.Potential('obs', logp) 


The last part is the part I described as "hacky" above. Since we are sampling a discrete variable to switch between models, we can't use the efficient NUTS sampler for all  of our variables, and `PyMC` will automatically switch to Metropolis for sampling `H_true`. (However, it's nice that `PyMC` _can_ sample from discrete variables and continuous variables in the same model; other popular probabalistic programming libraries can't do that.)

That's not going to be the only source of sampling inefficiency though:
> "Some common problems when computing Bayes factors this way is that if one model is better than the other, by definition, we will spend more time sampling from it than from the other model. And this could lead to inaccuracies because we will be undersampling the less likely model. Another problem is that the values of the parameters get updated even when the parameters are not used to fit that model . . . Therefore we end up having a problem with sampling." - [PyMC documentation](https://www.pymc.io/projects/examples/en/latest/diagnostics_and_criticism/Bayes_factor.html#using-a-hierarchical-model)

So to get a good posterior approximation for all three models (a requirement for a reliable Bayes Factor), one may have to tinker with the parameters of `pm.sample`. The parameters below worked for the current `pvals`, but might not work always. This is the main reason why I didn't include Bayes Factor computation in the `p2prev.PCurveMixture` API; no default sampler arguments are likely to work in the majority of cases. And at the point that you're successfully troubleshooting an MCMC sampler, you probably have a good enough grasp on the machinery of Bayesian model estimation that you don't need my API anymore. In any event, this will take much longer to sample than does the mixture model alone. 

In [3]:
with combined_model:
## Sample the model:
    #---------------------
    # MCMC sampler isn't inclined to visit low-probability
    # models frequently, so their parameters won't have 
    # half-decent posterior estimates unless we set 
    # `target_accept` to be super high
    target_accept = 0.99 
    # and we'll draw a really big amount
    # of samples since sampling efficiency
    # is so low....
    draws = 50000
    # but nothing else really needed to change
    idata = pm.sample(
        draws = draws, 
        chains = 5, 
        cores = 5, 
        random_seed = 0,
        target_accept = target_accept
    )

az.summary(idata)

Multiprocess sampling (5 chains in 5 jobs)
CompoundStep
>NUTS: [prevalence_mix, effect_size_H1, effect_size_mix]
>Metropolis: [H_true]


Output()

Sampling 5 chains for 1_000 tune and 50_000 draw iterations (5_000 + 250_000 draws total) took 510 seconds.


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
H_true[0],0.019,0.135,0.000,0.000,0.002,0.001,4129.0,4129.0,1.0
H_true[1],0.202,0.401,0.000,1.000,0.009,0.007,1799.0,1799.0,1.0
H_true[2],0.779,0.415,0.000,1.000,0.010,0.007,1762.0,1762.0,1.0
effect_size_H1,1.424,1.357,0.000,3.869,0.004,0.003,104764.0,62851.0,1.0
effect_size_mix,1.910,0.965,0.001,3.220,0.006,0.004,16059.0,10407.0,1.0
prevalence_mix,0.586,0.237,0.168,0.996,0.001,0.001,50327.0,23960.0,1.0


The HDIs of the `H_true` variables aren't really meaningful, since they can only take on values of 0 or 1 for each posterior sample. However, the approximate posterior probability of a given model (among the models considered) is given by the mean of the posterior samples of `H_true[model_index]`. You can then take the ratio of posterior probabilities of each model to get a Bayes Factor comparing them. 

In [4]:
# posterior odds of each hypothesis are Bayes factors
# if priors put equal wegiht on each model
# (which we did when we intialized `H_true` above)
H_true = idata.posterior.H_true.values
p_H0 = H_true[:, :, 0].mean()
p_H1 = H_true[:, :, 1].mean()
p_mix = H_true[:, :, 2].mean()
print('H0/H1 mixture vs. all H0: BF =', p_mix / p_H0)
print('H0/H1 mixture vs. all H1: BF =', p_mix / p_H1)
print('all H0 vs. all H1: BF =', p_H0 / p_H1)

H0/H1 mixture vs. all H0: BF = 42.03321829163072
H0/H1 mixture vs. all H1: BF = 3.858885500415858
all H0 vs. all H1: BF = 0.09180561606400255


>__Note: The posterior odds ratio for two models is only a valid Bayes Factor if the prior odds ratio for those models was equal to one.__ If, for some reason (e.g. to encourage the MCMC sampler to visit a poorly-fitting model more often), you didn't put equal priors on all models, you'll need to divide the posterior odds by the prior odds to get a Bayes Factor. _The above code will not always output a Bayes Factor without modification!_

If the Bayes Factor for the $H_0$/$H_1$ mixture vs. all-$H_0$ is 42, then this is interpreted as "the observed data were 42 times more likely under the mixture distribution than under the null hypothesis." Conversely, if a Bayes factor comparing all-$H_0$ vs. all-$H_1$ is 0.1, then the data are ten times _less_ likely under $H_0$ then they are under $H_1$. 

Note that the likelihood of the data given the model is not always the same as the posterior belief that one model is better than another. That, of course, depends on your prior over the models. In other words, the Bayes Factor is a measure of __strength of evidence__: how much one's prior should be updated, given the data. This quantity, usefully, is independent of ones prior over models, $\mathbb{P}(\text{model}_k)$, so it is an "objective" measure (in some sense) and thus a useful summary metric to share with other researchers who may have had different prior beliefs. 

> Note: Bayes Factors are not, however, independent of $\mathbb{P}(\text{data}|\text{model}_k, \theta_k)$, which depends on the priors over $\text{model}_k$'s parameters $\theta_k$ (e.g. like our $\text{Exponential}(\lambda = 1/1.5)$ prior we set for `effect_size_H1` and `effect_size_mix` in our model definition). In a very extreme case, if one were to set a prior such that $\mathbb{P}(\text{effect size} = 0) = 1$, then the $p$-curve under $H_1$ would be uniform (like under $H_0$) and all models would explain the data equally well. This is one reason to perform prior predictive simulations (see `prevalence-estimation.ipynb` for an example with the same prior we use here) so we know our priors aren't predicting anything weird.

### Final Note

In principle, one should be able to re-parametrize the above model to marginalize over `H_true` so that (1) all variable are continuous and (2) the likelihood of all three models are evaluated on each sampling step. (For example, see how the latent discrete variable in [this model](https://www.pymc.io/projects/examples/en/latest/generalized_linear_models/GLM-robust-with-outlier-detection.html#linear-model-with-custom-likelihood-to-distinguish-outliers-hogg-method) was re-parametrized into [this model](https://gist.github.com/dfm/5250dd2f17daf60cbe582ceeeb2fd12f), greatly improving sampling efficiency.) This would likely fix all the sampling issues I lamented above, but it is trickier than it looks, since the multinomial distribution of the discrete variable here is a _prior_ and thus needs to be sample-able for `pm.sample` to do its magic. This is a technical issue though, not a mathematical one (I think), so if anyone comes up with a solution please do let me know. 